# PI

In [64]:
import csv
import re
#from io import StringIO
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


In [53]:
#Funciones
#Separa columna tomada de logs en campos
def splitdata(col):
    temp = []
    for i in col:
        t = (i.split('='))
        temp.append(t[0])           
    return temp

#Normmaliza Dataset:
def splitdf(df):
    r = re.compile(r"[^a-zA-Z0-9-]")
    df2 = pd.DataFrame(columns=df.columns)
    for i in df:
        new = df[i].str.split('=',n=1, expand=True)
        df2[i] = new[1]#.str.extract(r'([a-zA-Z0-9=])')
        #df2[i] = r.sub("",new[1])
    return df2

In [54]:
#df = pd.read_csv('logs.csv')
df = pd.read_csv('botnet.csv')
lcol = [] 
lcol = df.columns
df.columns = splitdata(lcol)
df2 = splitdf(df)
df2
#[^=]*

Index(['itime', 'date', 'time', 'devid', 'vd', 'type', 'subtype', 'action',
       'app', 'appact', 'appcat', 'appid', 'applist', 'apprisk', 'apps', 'bid',
       'countapp', 'countweb', 'craction', 'crscore', 'devname', 'dstcountry',
       'dstintf', 'dstip', 'dstport', 'duration', 'dvid', 'hostname', 'level',
       'logid', 'logver', 'policyid', 'proto', 'rcvdbyte', 'rcvdpkt',
       'saasinfo', 'sentbyte', 'sentpkt', 'service', 'sessionid', 'srccountry',
       'srcintf', 'srcip', 'srcport', 'threatcnts', 'threatlvls', 'threats',
       'threattyps', 'threatwgts', 'trandisp', 'transip', 'transport',
       'utmaction', 'utmref'],
      dtype='object')

In [60]:
#Tuneo de Data Set
df2.head()

,itime,date,time,devid,vd,type,subtype,action,app,appact,...,threatcnts,threatlvls,threats,threattyps,threatwgts,trandisp,transip,transport,utmaction,utmref
0,1543765024,2018-12-02,12:37:03,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62114,block,BAQAAAAIAAAB3EAEyVgHI4B8Z_ANcchABMFYByOAfGfwDXA==
1,1543765017,2018-12-02,12:36:56,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62111,block,BAQAAAAIAAAB3EAHuVQGQ4B8S_ANcchAB7FUBkOAfEvwDXA==
2,1543765013,2018-12-02,12:36:53,FGT80C3913606343,root,traffic,forward,deny,Conficker.Botnet,drop-session,...,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62112,block,BAQAAAAIAAAB3EAH2VQGQ4B8S_ANcchAB9FUBkOAfEvwDXA==
3,1543765010,2018-12-02,12:36:49,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62110,block,BAQAAAAIAAAB3EAGyVQFY4B8L_ANcchABsFUBWOAfC_wDXA==
4,1543765009,2018-12-02,12:36:49,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62108,block,BAQAAAAIAAAB3EAGuVQFY4B8L_ANcchABrFUBUOAfCvwDXA==


In [61]:
df2.drop('utmref', axis=1, inplace = True)

In [66]:
train, test = train_test_split(df2, test_size = 0.3,random_state=1234)

In [67]:
print(train.shape)
print(test.shape)

(9, 53)
(5, 53)


In [68]:
train

,itime,date,time,devid,vd,type,subtype,action,app,appact,...,srcport,threatcnts,threatlvls,threats,threattyps,threatwgts,trandisp,transip,transport,utmaction
2,1543765013,2018-12-02,12:36:53,FGT80C3913606343,root,traffic,forward,deny,Conficker.Botnet,drop-session,...,1696,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62112,block
9,1543764987,2018-12-02,12:36:26,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,1682,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62098,block
7,1543764987,2018-12-02,12:36:26,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,1685,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62101,block
1,1543765017,2018-12-02,12:36:56,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,1695,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62111,block
8,1543764987,2018-12-02,12:36:26,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,1684,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62100,block
4,1543765009,2018-12-02,12:36:49,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,1692,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62108,block
5,1543765009,2018-12-02,12:36:49,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,1691,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62107,block
6,1543764995,2018-12-02,12:36:34,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,1688,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62104,block
3,1543765010,2018-12-02,12:36:49,FGT80C3913606343,root,traffic,forward,close,Conficker.Botnet,drop-session,...,1694,{1},{4},{Conficker.Botnet},{Botnet},{50},snat,200.41.178.10,62110,block
